In [49]:
import kagglehub

path = kagglehub.dataset_download("trolukovich/food11-image-dataset")
print("Dataset Path:", path)


Dataset Path: C:\Users\Yash Bade\.cache\kagglehub\datasets\trolukovich\food11-image-dataset\versions\1


In [54]:
import os
import random
import numpy as np
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
print("Root contents:", os.listdir(path))

TRAIN_DIR = os.path.join(path, "training")
VAL_DIR   = os.path.join(path, "validation")
TEST_DIR  = os.path.join(path, "evaluation")

IMG_SIZE = (160, 160)
BATCH_SIZE = 16
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen   = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_gen = val_datagen.flow_from_directory(
    VAL_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)
base_model = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(160, 160, 3)
)
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
output = Dense(train_gen.num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)
model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=2,
    steps_per_epoch=50,
    validation_steps=20,
    verbose=1
)
CALORIE_MAP = {
    "Bread": 265,
    "Dairy_product": 150,
    "Dessert": 300,
    "Egg": 155,
    "Fried_food": 350,
    "Meat": 250,
    "Noodles_Pasta": 220,
    "Rice": 200,
    "Seafood": 180,
    "Soup": 100,
    "Vegetable_Fruit": 80
}

CLASS_NAMES = list(train_gen.class_indices.keys())
def predict_food_and_calories(img_path):
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img = image.img_to_array(img) / 255.0
    img = np.expand_dims(img, axis=0)

    pred = model.predict(img, verbose=0)
    idx = np.argmax(pred)

    food = CLASS_NAMES[idx]
    calories = CALORIE_MAP.get(food, "Unknown")

    return food, calories
random_class = random.choice(os.listdir(TEST_DIR))
random_image = random.choice(os.listdir(os.path.join(TEST_DIR, random_class)))
test_image_path = os.path.join(TEST_DIR, random_class, random_image)

print("\nUsing test image:", test_image_path)

food, calories = predict_food_and_calories(test_image_path)

print("\n===== FINAL OUTPUT =====")
print("Predicted Food     :", food)
print("Estimated Calories :", calories, "kcal")


Root contents: ['evaluation', 'training', 'validation']
Found 9866 images belonging to 11 classes.
Found 3430 images belonging to 11 classes.
Epoch 1/2
50/50 ━━━━━━━━━━━━━━━━━━━━ 7s 100ms/step - accuracy: 0.5337 - loss: 1.4286 - val_accuracy: 0.6469 - val_loss: 0.9706
Epoch 2/2
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 76ms/step - accuracy: 0.7050 - loss: 0.8816 - val_accuracy: 0.7094 - val_loss: 0.8439

Using test image: C:\Users\Yash Bade\.cache\kagglehub\datasets\trolukovich\food11-image-dataset\versions\1\evaluation\Meat\85.jpg

===== FINAL OUTPUT =====
Predicted Food     : Meat
Estimated Calories : 250 kcal
